In [1]:
train_path = 'data/training.csv'
test_path = 'data/test.csv'
look_up_path = 'data/IdLookupTable.csv'

In [2]:
import pandas as pd
from PIL import Image, ImageDraw
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split
import csv


In [7]:
def print_image(image_ID):
    image_raw = []
    formatted_image = []
    for num in train_data['Image'][image_ID].split(' '):
        image_raw.append(int(num))

    for i in range(96):
        row = []
        for j in range(96):
            row.append(image_raw[i*96+j])
        formatted_image.append(row)

    img = Image.fromarray(np.array(formatted_image, dtype=np.uint8), mode='L').convert('RGB')
    draw = ImageDraw.Draw(img)
    point_columns = [col for col in train_data.columns if col != 'Image']
    points = train_data.iloc[0][point_columns].values.reshape(-1, 2)

    for point in points:
        draw.ellipse((point[0] - 1, point[1] - 1, point[0] + 1, point[1] + 1), fill='red', outline='red')
    img.show()

print_image(2)

NameError: name 'train_data' is not defined

# Data Cleanup 

Out of the 7,049 rows, 4,755 of these rows are missing 22 fields. We are going to average out those missing fields

In [32]:
# This ONLY includes rows that have data for at least 1 keypoint
# Rows that only contain image data are not included
train_data = pd.read_csv(train_path)

missing_image_rows = train_data['Image'].isnull().sum()
print(f"Rows missing 'Image' field: {missing_image_rows}\n")

missing_counts = train_data.isnull().sum(axis=1).value_counts().sort_index()
for missing_fields, row_count in missing_counts.items():
    if missing_fields > 0:
        print(f"Rows missing {missing_fields} field(s): {row_count}")

Rows missing 'Image' field: 0

Rows missing 2 field(s): 87
Rows missing 4 field(s): 28
Rows missing 6 field(s): 9
Rows missing 8 field(s): 8
Rows missing 10 field(s): 3
Rows missing 12 field(s): 5
Rows missing 14 field(s): 2
Rows missing 20 field(s): 2
Rows missing 22 field(s): 4755
Rows missing 24 field(s): 10


In [29]:
missing_row = train_data[train_data.isnull().any(axis=1)].iloc[0]
print(missing_row)

left_eye_center_x                                                     65.36512
left_eye_center_y                                                     39.98848
right_eye_center_x                                                    28.25152
right_eye_center_y                                                    37.39968
left_eye_inner_corner_x                                               59.32288
left_eye_inner_corner_y                                               40.85184
left_eye_outer_corner_x                                               71.98208
left_eye_outer_corner_y                                                41.7152
right_eye_inner_corner_x                                              33.71776
right_eye_inner_corner_y                                               38.5504
right_eye_outer_corner_x                                              21.34656
right_eye_outer_corner_y                                              37.97504
left_eyebrow_inner_end_x                            

In [35]:
column_means = {}

# Iterate over each column index except the last one
for i in range(train_data.shape[1] - 1):
    column_name = train_data.columns[i]
    mean_value = train_data.iloc[:, i].mean()
    column_means[i] = mean_value

avg_train = train_data.fillna(column_means)

# save train_data
# avg_train.to_csv('data/avg_train_data.csv', index=False)

missing_row = avg_train[avg_train.isnull().any(axis=1)].iloc[0]
print(missing_row)

In [30]:
column_means

{0: 66.35902124475365,
 1: 37.65123419193395,
 2: 30.306101514079753,
 3: 37.97694257172698,
 4: 59.15933943740387,
 5: 37.94475232796772,
 6: 73.3304779831461,
 7: 37.707008310792986,
 8: 36.65260694805725,
 9: 37.98990191451049,
 10: 22.384504479169355,
 11: 38.03349517525053,
 12: 56.06851307930929,
 13: 29.332678249311716,
 14: 79.48282586419953,
 15: 29.734860732432317,
 16: 39.32213692201747,
 17: 29.502998474112996,
 18: 15.871177472719152,
 19: 30.428165911873432,
 20: 48.37418629032876,
 21: 62.71588359736171,
 22: 63.285735120986686,
 23: 75.97071433583544,
 24: 32.90040370265762,
 25: 76.17976785460499,
 26: 47.9754115720203,
 27: 72.91944259683736,
 28: 48.56947217227819,
 29: 78.97014624503687}

7049
7049
